In [1]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd

from scipy.io import arff
data = arff.loadarff('wq.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:14].values
response = df.iloc[:,14:30].values
positions = np.arange(1060)

from sklearn.model_selection import train_test_split

covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#import tensorflow as tf 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,targets_train.shape[1]])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,targets_train.shape[1]])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, targets_train.shape[1]]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 25 #200
M = 16 #160
#N = 100
#O = 30

# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
#W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
#B3 = tf.Variable(tf.ones([N])/10)
#W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
#B4 = tf.Variable(tf.ones([O])/10)
#W5 = tf.Variable(tf.truncated_normal([O, 6], stddev=0.1))
#B5 = tf.Variable(tf.zeros([6]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

#Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
#Y2d = tf.nn.dropout(Y2, pkeep)

#Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
#Y3d = tf.nn.dropout(Y3, pkeep)

#Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
#Y4d = tf.nn.dropout(Y4, pkeep)

#Y_ = tf.matmul(Y4d, W5) + B5


Y_ = tf.matmul(Y1d, W2) + B2

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)
# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(3500):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
LOSS at step 0: - Train loss value : 3.216738 -- Test loss value : 2.765843
LOSS at step 0: - Train loss value : 2.496445 -- Test loss value : 2.759832
LOSS at step 0: - Train loss value : 2.493032 -- Test loss value : 2.753829
LOSS at step 0: - Train loss value : 2.530670 -- Test loss value : 2.747847
LOSS at step 0: - Train loss value : 2.499119 -- Test loss value : 2.741936
LOSS at step 0: - Train loss value : 2.643059 -- Test loss value : 2.736056
LOSS at step 0: - Train loss value : 2.408253 -- Test loss value : 2.730197
LOSS at step 0: - Train loss value : 2.910674 -- Test loss value : 2.724305
LOSS at step 0: - Train loss value : 2.684307 -- Test loss value : 2.718421
LOSS at step 0: - Train loss value : 2.615194 -- Test loss value : 2.712592
LOSS at step 0: - Train loss value : 3.4

LOSS at step 140: - Train loss value : 1.579678 -- Test loss value : 1.509255
LOSS at step 140: - Train loss value : 1.383619 -- Test loss value : 1.509241
LOSS at step 140: - Train loss value : 1.330767 -- Test loss value : 1.509221
LOSS at step 140: - Train loss value : 1.423002 -- Test loss value : 1.509203
LOSS at step 140: - Train loss value : 1.414951 -- Test loss value : 1.509184
LOSS at step 140: - Train loss value : 1.476340 -- Test loss value : 1.509173
LOSS at step 140: - Train loss value : 1.468068 -- Test loss value : 1.509152
LOSS at step 140: - Train loss value : 1.640813 -- Test loss value : 1.509108
LOSS at step 140: - Train loss value : 1.321855 -- Test loss value : 1.509079
LOSS at step 140: - Train loss value : 1.126187 -- Test loss value : 1.509055
LOSS at step 140: - Train loss value : 1.366172 -- Test loss value : 1.509017
LOSS at step 140: - Train loss value : 1.193188 -- Test loss value : 1.509009
LOSS at step 140: - Train loss value : 1.687024 -- Test loss val

LOSS at step 280: - Train loss value : 1.072618 -- Test loss value : 1.481267
LOSS at step 280: - Train loss value : 1.480359 -- Test loss value : 1.481305
LOSS at step 280: - Train loss value : 1.217752 -- Test loss value : 1.481328
LOSS at step 280: - Train loss value : 1.273618 -- Test loss value : 1.481354
LOSS at step 280: - Train loss value : 1.749302 -- Test loss value : 1.481375
LOSS at step 280: - Train loss value : 1.342603 -- Test loss value : 1.481400
LOSS at step 280: - Train loss value : 1.401706 -- Test loss value : 1.481423
LOSS at step 280: - Train loss value : 1.226849 -- Test loss value : 1.481441
LOSS at step 280: - Train loss value : 1.510257 -- Test loss value : 1.481444
LOSS at step 280: - Train loss value : 1.566348 -- Test loss value : 1.481415
LOSS at step 280: - Train loss value : 1.288036 -- Test loss value : 1.481393
LOSS at step 280: - Train loss value : 1.641996 -- Test loss value : 1.481358
LOSS at step 280: - Train loss value : 1.415743 -- Test loss val

LOSS at step 420: - Train loss value : 1.255204 -- Test loss value : 1.472408
LOSS at step 420: - Train loss value : 1.198901 -- Test loss value : 1.472385
LOSS at step 420: - Train loss value : 1.580953 -- Test loss value : 1.472332
LOSS at step 420: - Train loss value : 1.570526 -- Test loss value : 1.472294
LOSS at step 420: - Train loss value : 1.070979 -- Test loss value : 1.472284
LOSS at step 420: - Train loss value : 1.318365 -- Test loss value : 1.472261
LOSS at step 420: - Train loss value : 1.574919 -- Test loss value : 1.472234
LOSS at step 420: - Train loss value : 1.173369 -- Test loss value : 1.472210
LOSS at step 420: - Train loss value : 1.366042 -- Test loss value : 1.472188
LOSS at step 420: - Train loss value : 1.555983 -- Test loss value : 1.472153
LOSS at step 420: - Train loss value : 1.433006 -- Test loss value : 1.472125
LOSS at step 420: - Train loss value : 1.563264 -- Test loss value : 1.472072
LOSS at step 420: - Train loss value : 1.286479 -- Test loss val

LOSS at step 560: - Train loss value : 1.264385 -- Test loss value : 1.462850
LOSS at step 560: - Train loss value : 1.096862 -- Test loss value : 1.462883
LOSS at step 560: - Train loss value : 1.452408 -- Test loss value : 1.462914
LOSS at step 560: - Train loss value : 1.733810 -- Test loss value : 1.462923
LOSS at step 560: - Train loss value : 1.785749 -- Test loss value : 1.462877
LOSS at step 560: - Train loss value : 1.383369 -- Test loss value : 1.462821
LOSS at step 560: - Train loss value : 1.473144 -- Test loss value : 1.462725
LOSS at step 560: - Train loss value : 1.436640 -- Test loss value : 1.462651
LOSS at step 560: - Train loss value : 1.115162 -- Test loss value : 1.462614
LOSS at step 560: - Train loss value : 1.289486 -- Test loss value : 1.462599
LOSS at step 560: - Train loss value : 1.218463 -- Test loss value : 1.462605
LOSS at step 560: - Train loss value : 1.633642 -- Test loss value : 1.462586
LOSS at step 560: - Train loss value : 1.604370 -- Test loss val

...

LOSS at step 2800: - Train loss value : 1.642783 -- Test loss value : 1.397739
LOSS at step 2800: - Train loss value : 1.151075 -- Test loss value : 1.397706
LOSS at step 2800: - Train loss value : 1.243870 -- Test loss value : 1.397686
LOSS at step 2800: - Train loss value : 1.510501 -- Test loss value : 1.397674
LOSS at step 2800: - Train loss value : 0.988133 -- Test loss value : 1.397680
LOSS at step 2800: - Train loss value : 1.390690 -- Test loss value : 1.397665
LOSS at step 2800: - Train loss value : 1.290948 -- Test loss value : 1.397657
LOSS at step 2800: - Train loss value : 1.065135 -- Test loss value : 1.397654
LOSS at step 2800: - Train loss value : 1.646235 -- Test loss value : 1.397626
LOSS at step 2800: - Train loss value : 1.339548 -- Test loss value : 1.397591
LOSS at step 2800: - Train loss value : 1.315943 -- Test loss value : 1.397537
LOSS at step 2800: - Train loss value : 1.432504 -- Test loss value : 1.397486
LOSS at step 2800: - Train loss value : 1.530275 -- 

LOSS at step 2940: - Train loss value : 1.345989 -- Test loss value : 1.397781
LOSS at step 2940: - Train loss value : 1.402691 -- Test loss value : 1.397809
LOSS at step 2940: - Train loss value : 1.237747 -- Test loss value : 1.397853
LOSS at step 2940: - Train loss value : 1.607875 -- Test loss value : 1.397892
LOSS at step 2940: - Train loss value : 1.398530 -- Test loss value : 1.397933
LOSS at step 2940: - Train loss value : 1.104764 -- Test loss value : 1.397985
LOSS at step 2940: - Train loss value : 0.992768 -- Test loss value : 1.398065
LOSS at step 2940: - Train loss value : 1.317158 -- Test loss value : 1.398148
LOSS at step 2940: - Train loss value : 1.579848 -- Test loss value : 1.398231
LOSS at step 2940: - Train loss value : 1.514598 -- Test loss value : 1.398311
LOSS at step 2940: - Train loss value : 1.555523 -- Test loss value : 1.398366
LOSS at step 2940: - Train loss value : 1.279749 -- Test loss value : 1.398401
LOSS at step 2940: - Train loss value : 1.202302 -- 

LOSS at step 3080: - Train loss value : 1.555038 -- Test loss value : 1.395465
LOSS at step 3080: - Train loss value : 1.150292 -- Test loss value : 1.395457
LOSS at step 3080: - Train loss value : 1.462109 -- Test loss value : 1.395469
LOSS at step 3080: - Train loss value : 1.325901 -- Test loss value : 1.395486
LOSS at step 3080: - Train loss value : 1.052590 -- Test loss value : 1.395516
LOSS at step 3080: - Train loss value : 0.960289 -- Test loss value : 1.395547
LOSS at step 3080: - Train loss value : 1.449347 -- Test loss value : 1.395555
LOSS at step 3080: - Train loss value : 1.190223 -- Test loss value : 1.395561
LOSS at step 3080: - Train loss value : 1.354880 -- Test loss value : 1.395570
LOSS at step 3080: - Train loss value : 1.328425 -- Test loss value : 1.395581
LOSS at step 3080: - Train loss value : 1.355361 -- Test loss value : 1.395574
LOSS at step 3080: - Train loss value : 1.135111 -- Test loss value : 1.395569
LOSS at step 3080: - Train loss value : 1.287138 -- 

LOSS at step 3220: - Train loss value : 1.380346 -- Test loss value : 1.393686
LOSS at step 3220: - Train loss value : 1.246637 -- Test loss value : 1.393656
LOSS at step 3220: - Train loss value : 1.224972 -- Test loss value : 1.393641
LOSS at step 3220: - Train loss value : 1.165662 -- Test loss value : 1.393654
LOSS at step 3220: - Train loss value : 1.228786 -- Test loss value : 1.393677
LOSS at step 3220: - Train loss value : 1.448990 -- Test loss value : 1.393706
LOSS at step 3220: - Train loss value : 1.100388 -- Test loss value : 1.393736
LOSS at step 3220: - Train loss value : 1.053177 -- Test loss value : 1.393781
LOSS at step 3220: - Train loss value : 1.352061 -- Test loss value : 1.393810
LOSS at step 3220: - Train loss value : 1.641794 -- Test loss value : 1.393832
LOSS at step 3220: - Train loss value : 1.408268 -- Test loss value : 1.393864
LOSS at step 3220: - Train loss value : 1.477319 -- Test loss value : 1.393909
LOSS at step 3220: - Train loss value : 1.344287 -- 

LOSS at step 3360: - Train loss value : 1.496856 -- Test loss value : 1.393833
LOSS at step 3360: - Train loss value : 1.210018 -- Test loss value : 1.393842
LOSS at step 3360: - Train loss value : 1.498190 -- Test loss value : 1.393815
LOSS at step 3360: - Train loss value : 1.242977 -- Test loss value : 1.393802
LOSS at step 3360: - Train loss value : 1.195864 -- Test loss value : 1.393795
LOSS at step 3360: - Train loss value : 0.903442 -- Test loss value : 1.393805
LOSS at step 3360: - Train loss value : 0.910789 -- Test loss value : 1.393804
LOSS at step 3360: - Train loss value : 1.493617 -- Test loss value : 1.393801
LOSS at step 3360: - Train loss value : 1.098819 -- Test loss value : 1.393806
LOSS at step 3360: - Train loss value : 1.380871 -- Test loss value : 1.393802
LOSS at step 3360: - Train loss value : 1.181074 -- Test loss value : 1.393798
LOSS at step 3360: - Train loss value : 1.364237 -- Test loss value : 1.393769
LOSS at step 3360: - Train loss value : 1.316846 -- 

# Let us compute the aRRMSE

In [3]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [4]:
Y_testpred

array([[0.26167223, 0.1451693 , 0.66392696, ..., 0.6145944 , 0.7092289 ,
        0.3475385 ],
       [0.39987466, 0.1956468 , 1.0554631 , ..., 0.45175725, 0.9756104 ,
        0.604798  ],
       [0.82799137, 0.6168226 , 0.31133807, ..., 0.3551532 , 0.5325718 ,
        1.0467153 ],
       ...,
       [0.29648817, 0.15591873, 1.2508054 , ..., 0.55962133, 1.0819979 ,
        0.3114775 ],
       [0.6823151 , 0.46303236, 0.47182027, ..., 0.29018733, 0.45475143,
        1.1959624 ],
       [0.34437668, 0.17525831, 0.7323568 , ..., 0.46425754, 0.95629495,
        0.46342215]], dtype=float32)

In [5]:
from numpy import linalg as LA

In [6]:

aRRMSE_list = []

for i in range(Y_testpred.shape[1]) :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 4.434021
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 4.764626
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 6.500748
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 6.735078
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 6.813510
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 6.106534
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 6.654508
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 6.206888
<*******

In [7]:
np.mean(aRRMSE_list)

6.273671659419552

In [9]:
#Taking aRRMSE on PCA or inverted PCA. ?

# Check if old way calculation is wrong

In [10]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 82.647642


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [11]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [12]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [10]:
spnnr_aRRMSE_list = []

for i in range(spnnr_Y_testpred.shape[1]) :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.212108
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.784301
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.862991


In [11]:
np.mean(spnnr_aRRMSE_list)

15.286466834678842